In [17]:
import dill as pickle
import pandas as pd

In [18]:
ls ../data/act_inh_pkl

231003-b0723-greedy.pkl  231003-b1101-greedy.pkl  231003-b2239-greedy.pkl
231003-b0723.pkl         231003-b1101.pkl         231003-b2239.pkl


In [25]:
pickle_in = open('../data/act_inh_pkl/231003-b2239-greedy.pkl', 'rb')
a = pickle.load(pickle_in)

In [26]:
for val in a[0]:
    print(val)

3.630780547701014
2.884031503126606
1.7868012422392947
8.31763771102671
0.02290299101575983
0.32274567015748645
8.766983113552968
4.36515832240166
3.311311214825911
1.4454397707459274
5.011872336272724
1.4454397707459274
0.004931594317965625
2.423423603095939
2.4975121157159146
2.3988329190194904
9.120108393559097
4.168693834703354
1.0471285480508996
1.9952623149688795
1.096478196143185
0.07385234756199617
8.709635899560807
1.9952623149688795
1.0471285480508996
1.318256738556407
1.096478196143185
8.571910962764187
0.00407336746012933
0.0022547594561292965
0.20602318628389196
0.0012809129386032127
8.709635899560807
9.052135781779484
2.2194078173537797
2.850924674043761
1.390342388363182
2.0892961308540396
2.290867652767773
2.129391655928126
1.6595869074375607
0.6094688623440997
0.30150810794346394
2.884031503126606
0.0792939281823341
0.23070761952640131
0.0746842968610376
0.07511780454562579
1.9054607179632472
0.21763556090210393
0.009746605341139183
0.009993357967132988
0.0089877547690

In [10]:
a[1]

[158.48931924611142,
 115.10260045756357,
 467.72430577572993,
 367.20595561981486,
 131.56826064833706,
 130.69082724521732,
 398.1071705534973,
 364.5040313138397,
 304.9597388773301,
 177.82794100389228,
 161.63941866040753,
 159.8414720813878,
 83.4414405634976,
 158.66437984592878,
 160.3061133241512,
 158.48931924611142,
 163.08844253608459,
 141.47921883779551,
 177.82794100389228,
 251.18864315095823,
 177.82794100389228,
 155.85897002509537,
 354.81338923357566,
 253.3998178395233,
 199.52623149688807,
 187.87157649207367,
 188.574778498691,
 195.286012302152,
 0.28183829312644537,
 63.095734448019364,
 141.25375446227554,
 1.0,
 255.80017843862316,
 420.6906174144198,
 179.45618895484083,
 160.76100389622357,
 158.48931924611142,
 186.8513149655894,
 138.07200998509828,
 172.508936760998,
 178.54722404619707,
 137.92873585919483,
 132.3606786017897,
 140.14789569838922,
 112.28132146803696,
 124.30716203302838,
 141.25375446227554,
 138.69386318339315,
 181.79741279213636,
 1

In [11]:
a[2]

['control__wt_glc__1',
 'control__wt_glc__2',
 'fur__wt_dpd__1',
 'fur__wt_dpd__2',
 'fur__wt_fe__1',
 'fur__wt_fe__2',
 'fur__delfur_dpd__1',
 'fur__delfur_dpd__2',
 'fur__delfur_fe2__1',
 'fur__delfur_fe2__2',
 'acid__wt_ph5__1',
 'acid__wt_ph5__2',
 'acid__delgadx_ph5__1',
 'acid__delgadx_ph5__2',
 'acid__delgadw_ph5__1',
 'acid__delgadw_ph5__2',
 'acid__delgade_ph5__1',
 'acid__delgade_ph5__2',
 'oxidative__wt_pq__1',
 'oxidative__wt_pq__2',
 'oxidative__deloxyr_pq__1',
 'oxidative__deloxyr_pq__2',
 'oxidative__delsoxr_pq__1',
 'oxidative__delsoxr_pq__2',
 'oxidative__delsoxs_pq__1',
 'oxidative__delsoxs_pq__2',
 'nac_ntrc__wt_csn__1',
 'nac_ntrc__wt_csn__2',
 'nac_ntrc__wt_cytd__1',
 'nac_ntrc__wt_cytd__2',
 'nac_ntrc__wt_gln__1',
 'nac_ntrc__wt_gln__2',
 'nac_ntrc__bw_delnac_csn__1',
 'nac_ntrc__bw_delnac_csn__2',
 'nac_ntrc__bw_delnac_cytd__1',
 'nac_ntrc__bw_delnac_cytd__2',
 'nac_ntrc__bw_delnac_gln__1',
 'nac_ntrc__bw_delnac_gln__2',
 'nac_ntrc__bw_delnac_nh4__1',
 'nac_ntrc_

In [12]:
a[3]

(0.8952992021507751, -0.9399409588017691)

In [13]:
a[4]

{'KdRNAP': 1e-05,
 'KdRNAPCrp': 3.516641004113411e-07,
 'KeqOpening': 0.4524289955987184,
 'RNAP': 1e-06}

In [14]:
a[5]

{'seed': 42,
 'n_gen': 1000,
 'n_ind': 100,
 'n_iter': 10,
 'mu': 100,
 'lambda_': 100,
 'cx_ind': 1,
 'cx_gene': 0.6,
 'mt_ind': 0,
 'mt_gene': 0.1}

In [15]:
a[6]

IndexError: list index out of range